<a href="https://colab.research.google.com/github/Mageed-Ghaleb/OptimizationSystems-Course/blob/main/Lab%2001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 1 — Systems Modeling & Tooling (Optimization Systems)

**Course:** Optimization Systems (Graduate)  
**Lecture:** 1 — Systems Modeling & Tooling  

**Goals:**

- Get a working Google Colab environment for this course.
- Build and solve a tiny *capacity expansion* model in Pyomo.
- Inspect solver logs and the optimal solution.
- Perform a simple sensitivity experiment (e.g., demand +10%).
- (Optional) See a tiny assignment example in OR-Tools CP-SAT.

> Always run the installation cell **first** when you open this notebook.

In [4]:
# === Install required packages (run this cell first) ===
!pip install -q "protobuf<6" pyomo ortools

## 1. Imports and Data

We will define a tiny capacity expansion instance using pandas DataFrames.

We have:
- A set of **facilities** with fixed costs and capacities.
- A set of **regions** with demands.
- A **shipping cost** from each facility to each region.

In [3]:
import pandas as pd

from pyomo.environ import (
    ConcreteModel, Var, Objective, Constraint,
    NonNegativeReals, Binary, minimize, value, Set
)
from pyomo.opt import SolverFactory

In [4]:
# --- Toy data inline (you could later replace with CSVs) ---

facilities = pd.DataFrame({
    "id": ["F1", "F2", "F3"],
    "fixed_cost": [5000, 4000, 4500],
    "capacity": [120, 100, 110],
})

regions = pd.DataFrame({
    "id": ["R1", "R2", "R3"],
    "demand": [80, 90, 70],
})

ship_cost = pd.DataFrame(
    [
        ("F1", "R1", 2), ("F1", "R2", 3), ("F1", "R3", 4),
        ("F2", "R1", 3), ("F2", "R2", 2), ("F2", "R3", 3),
        ("F3", "R1", 4), ("F3", "R2", 3), ("F3", "R3", 2),
    ],
    columns=["facility_id", "region_id", "unit_cost"]
)

facilities, regions, ship_cost

(   id  fixed_cost  capacity
 0  F1        5000       120
 1  F2        4000       100
 2  F3        4500       110,
    id  demand
 0  R1      80
 1  R2      90
 2  R3      70,
   facility_id region_id  unit_cost
 0          F1        R1          2
 1          F1        R2          3
 2          F1        R3          4
 3          F2        R1          3
 4          F2        R2          2
 5          F2        R3          3
 6          F3        R1          4
 7          F3        R2          3
 8          F3        R3          2)

## 2. Pyomo Model: Capacity Expansion

We now:

1. Create sets and parameters from the tables.  
2. Create decision variables:
   - `open[f]` ∈ {0,1} — facility open decision
   - `x[f,r]` ≥ 0 — shipped units  
3. Add:
   - Demand satisfaction constraints
   - Capacity linking constraints  
4. Minimize fixed + shipping cost.

In [5]:
m = ConcreteModel()

m.F = Set(initialize=list(facilities["id"]))
m.R = Set(initialize=list(regions["id"]))

fixed = {row.id: float(row.fixed_cost) for _, row in facilities.iterrows()}
cap = {row.id: float(row.capacity) for _, row in facilities.iterrows()}
dem = {row.id: float(row.demand) for _, row in regions.iterrows()}
c = {(r.facility_id, r.region_id): float(r.unit_cost) for _, r in ship_cost.iterrows()}

m.open = Var(m.F, within=Binary)
m.x = Var(m.F, m.R, within=NonNegativeReals)

def obj_rule(m):
    return sum(fixed[f] * m.open[f] for f in m.F) + \
           sum(c[(f, r)] * m.x[(f, r)] for f in m.F for r in m.R)

m.OBJ = Objective(rule=obj_rule, sense=minimize)

def demand_rule(m, r):
    return sum(m.x[(f, r)] for f in m.F) >= dem[r]
m.Demand = Constraint(m.R, rule=demand_rule)

def capacity_rule(m, f):
    return sum(m.x[(f, r)] for r in m.R) <= cap[f] * m.open[f]
m.Capacity = Constraint(m.F, rule=capacity_rule)

m.pprint()

2 Set Declarations
    F : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'F1', 'F2', 'F3'}
    R : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'R1', 'R2', 'R3'}

2 Var Declarations
    open : Size=3, Index=F
        Key : Lower : Value : Upper : Fixed : Stale : Domain
         F1 :     0 :  None :     1 : False :  True : Binary
         F2 :     0 :  None :     1 : False :  True : Binary
         F3 :     0 :  None :     1 : False :  True : Binary
    x : Size=9, Index=F*R
        Key          : Lower : Value : Upper : Fixed : Stale : Domain
        ('F1', 'R1') :     0 :  None :  None : False :  True : NonNegativeReals
        ('F1', 'R2') :     0 :  None :  None : False :  True : NonNegativeReals
        ('F1', 'R3') :     0 :  None :  None : False :  True : NonNegativeReals
        ('F2', 'R1') :     0 :  None :  None : False

## 3. Solve the Model and Inspect the Solution

We will use the open-source **HiGHS** solver (via `highspy`) on Colab.

If the solver fails for any reason, note the error and discuss with your instructor.

In [6]:
solver = SolverFactory("highs")
results = solver.solve(m, tee=True)  # tee=True prints solver log

print("\n===== Solution Summary =====")
print("Status:", results.solver.status)
print("Termination condition:", results.solver.termination_condition)

print("Total cost:", value(m.OBJ))
print("Opened facilities:", [f for f in m.F if m.open[f].value > 0.5])

print("\nShipments (only positive flows):")
for f in m.F:
    for r in m.R:
        val = m.x[(f, r)].value
        if val and val > 1e-6:
            print(f"  {f} -> {r}: {val:.2f}")

Running HiGHS 1.12.0 (git hash: 755a8e0): Copyright (c) 2025 HiGHS under MIT licence terms
MIP has 6 rows; 12 cols; 21 nonzeros; 3 integer variables (3 binary)
Coefficient ranges:
  Matrix  [1e+00, 1e+02]
  Cost    [2e+00, 5e+03]
  Bound   [1e+00, 1e+00]
  RHS     [7e+01, 9e+01]
Presolving model
6 rows, 12 cols, 21 nonzeros  0s
6 rows, 12 cols, 21 nonzeros  0s
Presolve reductions: rows 6(-0); columns 12(-0); nonzeros 21(-0) - Not reduced

Solving MIP model with:
   6 rows
   12 cols (3 binary, 0 integer, 0 implied int., 9 continuous, 0 domain fixed)
   21 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic;
     I => Shifting; J => Feasibility jump; L => Sub-MIP; P => Empty MIP; R => Randomized rounding;
     S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution; Y => HiGHS solution;
     Z => ZI Round; l => Trivial lower; p => Trivial point; u => Trivial upper; z => Trivial zero

        Nodes      |    B&B Tree     |            

### Quick Questions

- Which facilities opened? Why do you think the model chose them?
- Are any facilities open but barely used?
- Which constraints do you expect to be binding at optimum:
  - Capacity constraints?
  - Demand constraints?
  - Both?

## 4. Sensitivity: Demand +10%

Now we increase all demands by 10% and re-solve.

We keep the same model, just change the right-hand side of the demand constraints.

In [7]:
# Increase demand by 10%
dem_up = {k: v * 1.10 for k, v in dem.items()}
for r in m.R:
    m.Demand[r].set_value(sum(m.x[(f, r)] for f in m.F) >= dem_up[r])

results_up = solver.solve(m, tee=False)

print("===== After +10% demand =====")
print("Status:", results_up.solver.status)
print("Termination condition:", results_up.solver.termination_condition)
print("Total cost:", value(m.OBJ))
print("Opened facilities:", [f for f in m.F if m.open[f].value > 0.5])

===== After +10% demand =====
Status: ok
Termination condition: optimal
Total cost: 14028.0
Opened facilities: ['F1', 'F2', 'F3']


> **Think about:**
> - Did the set of open facilities change?
> - By roughly how much did the total cost increase?
> - Did any new constraints become binding?
>
> These observations will help when we talk about **duality** and **sensitivity analysis** later.

## 5. (Optional) OR-Tools CP-SAT: Tiny Assignment Example

This is a small example of an assignment problem using OR-Tools CP-SAT.

Later in the course we will use CP-SAT more seriously for scheduling and routing.

In [8]:
from ortools.sat.python import cp_model

cost = [
    [14, 5, 8],
    [7,  8, 3],
    [2,  6, 5],
]

n_workers = len(cost)
n_tasks = len(cost[0])

model = cp_model.CpModel()

x = {}
for i in range(n_workers):
    for j in range(n_tasks):
        x[i, j] = model.NewBoolVar(f"x[{i},{j}]")

# Each worker at most one task
for i in range(n_workers):
    model.Add(sum(x[i, j] for j in range(n_tasks)) <= 1)

# Each task exactly one worker
for j in range(n_tasks):
    model.Add(sum(x[i, j] for i in range(n_workers)) == 1)

model.Minimize(sum(cost[i][j] * x[i, j] for i in range(n_workers) for j in range(n_tasks)))

solver_cp = cp_model.CpSolver()
solver_cp.parameters.max_time_in_seconds = 5

status = solver_cp.Solve(model)
print("Status:", solver_cp.StatusName(status))
print("Objective value:", solver_cp.ObjectiveValue())

for i in range(n_workers):
    for j in range(n_tasks):
        if solver_cp.Value(x[i, j]) == 1:
            print(f"Worker {i} -> Task {j}")

Status: OPTIMAL
Objective value: 10.0
Worker 0 -> Task 1
Worker 1 -> Task 2
Worker 2 -> Task 0


## 6. Lecture 1 Deliverable — Short Memo on Model-Data Contract

Using this notebook and Lecture 1, write a **1-2 page memo** describing the model–data contract for the capacity expansion problem:

- What tables (and columns) does the model expect?
- How do these map to sets, parameters, and constraints?
- What assumptions are you making (demands, capacities, time horizon)?
- How would you structure files and scenarios (e.g., baseline vs +10% demand)?

Follow the memo instructions provided by your instructor.